<a href="https://colab.research.google.com/github/Oxcitzen/GOES-R-Python-Script/blob/main/GOES_R_Access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output 

In [ ]:
# !pip install GEOS
# !pip install Shapely
# !pip install pyshp
# !pip install Cython
# !pip install six
# !pip install cartopy
# !pip install pyproj
# !pip install geos
# !pip install geoviews
# !pip install metpy
!pip install goes2go
!pip install zarr
clear_output()

In [ ]:
def group_event(A, B, elem):
    idx = np.where(A == elem)[0]
    first_idx = idx[0]
    last_idx = idx[-1]
    return B.tolist()[first_idx:last_idx+1]
    
def find_heirarchy(df): 
  group_under_flash = df['group_parent_flash_id'].to_numpy()
  group_ids = df['group_id'].to_numpy()

  event_under_group = df['event_parent_group_id'].to_numpy()
  event_ids = df['event_id'].to_numpy()

  total_dict = {} 

  for i in range(len(group_under_flash)):
        if group_under_flash[i] not in total_dict:
            total_dict[group_under_flash[i]] = {}
        total_dict[group_under_flash[i]][group_ids[i]] = group_event(event_under_group, event_ids, group_ids[i])
  return total_dict

def count(dic, c=0):
  for key in dic: 
    if isinstance(dic[key], list): 
      c += len(dic[key])
    else: 
      c = count(dic[key], c)
  return c

def finalize_GLM(df): 
  vars = list(df.variables)
  del_vars = [variable for variable in vars if variable not in ['event_id', 'event_lat', 'event_lon', 'lat_field_of_view_bounds', 'lon_field_of_view_bounds', 'event_time_offset']]
  final = df.drop_vars(del_vars)

  latitudes = df.event_lat.to_numpy()
  longitudes = df.event_lon.to_numpy()
  event_id = df.event_id.to_numpy()

  long_ind = (longitudes >=-125) * (longitudes<= -115)
  lat_ind = (latitudes >=30) * (latitudes<= 55)

  inside = np.logical_and(long_ind, lat_ind)
  
  final = final.assign(event_lat = latitudes[inside])
  final = final.assign(event_lon = longitudes[inside])
  final = final.assign(event_id = event_id[inside])
  return final 

def saveCube(xarray, name, location, local_storage_check): 
  print("This will take a while! For larger datasets, you might have to wait up 7-10 minutes.\n")
  if local_storage_check: 
    absolute_path = os.path.abspath(f'../datasets/{name}')
  else: 
    absolute_path = location + name
  xarray.to_netcdf(path = absolute_path, mode = "w", format = "NETCDF4")
  print(f"🎉 File download completed! 🎉\nName: {name}\nLocation: {location}")

#def crop_strikes(df, coords): 


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

from goes2go import GOES
from goes2go.data import goes_timerange
import dask.array
import csv, os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import xarray as xr
import zarr
import netCDF4

# Products
# GLM -> Lightning
# ABI -> Cloud coverage

# Domains
# C -> CONUS (Americas)
# F -> Full (East and West)

# Latest 
# df = GOES(satellite=16, product="GLM-L2-LCFA", domain='C').latest()
# df

# Multiple Days
# df_1_2 = goes_timerange(start='2022-01-02 00:00', end='2022-01-03 00:00', satellite='goes16', product='GLM', domain='C', return_as='xarray', download = True, save_dir = "/content/saves")

In [ ]:
# For GOES17 data, make sure to put it in UTC datatime format
# Use this -> goes_timerange(start ="2022-06-22T00:00:00", end ="2022-06-24T00:00:00", satellite='goes17', product='GLM', domain='C', download = True, save_dir  = "/content/drive/MyDrive/lwfShare/Datasets/DS_Team_Trial/goes_r/cali_goes/") 

In [ ]:
location = "/content/drive/MyDrive/lwfShare/Datasets/DS_Team_Trial/goes_r/cali_goes/noaa-goes17/GLM-L2-LCFA/2022/173/09/OR_GLM-L2-LCFA_G17_s20221730900000_e20221730900200_c20221730900214.nc"
df = xr.open_dataset(location, chunks={"number_of_events": 1000})

In [ ]:
df = finalize_GLM(df)

IndexError: ignored

In [ ]:
df

<xarray.Dataset>
Dimensions:                   (number_of_events: 1480, event_lat: 483,
                               event_lon: 483, number_of_field_of_view_bounds: 2)
Coordinates:
    event_id                  (number_of_events) uint32 dask.array<chunksize=(1000,), meta=np.ndarray>
    event_time_offset         (number_of_events) datetime64[ns] dask.array<chunksize=(1000,), meta=np.ndarray>
  * event_lat                 (event_lat) float64 34.54 34.46 ... 34.45 34.45
  * event_lon                 (event_lon) float64 -118.1 -118.1 ... -118.3
Dimensions without coordinates: number_of_events, number_of_field_of_view_bounds
Data variables:
    lat_field_of_view_bounds  (number_of_field_of_view_bounds) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    lon_field_of_view_bounds  (number_of_field_of_view_bounds) float32 dask.array<chunksize=(2,), meta=np.ndarray>
Attributes: (12/29)
    production_site:           WCDAS
    featureType:               point
    dataset_name:              OR_GLM-L2-LCFA_G17_s20221730900000_e2022173090...
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    institution:               DOC/NOAA/NESDIS> U.S. Department of Commerce, ...
    ...                        ...
    time_coverage_start:       2022-06-22T09:00:00.0Z
    time_coverage_end:         2022-06-22T09:00:20.0Z
    production_data_source:    Realtime
    production_environment:    OE
    LUT_Filenames:             GLM_CALINR_AllFilters(FM2W_CDRL79RevD_PR_09_00...
    id:                        fe5693b0-e869-4a1a-a8db-aa9d64d5ddd2

array([-66.49731019, -66.49731019, -66.49731019, ..., -66.48966453,
       -66.48969341, -66.48966453])

In [ ]:
def is_inside(boundaries, point):
  # Boundaries must be a Polygon, going counterclockwise
  # Point must be a vec 

  AM = np.subtract(point, boundaries[0])
  AB = np.subtract(boundaries[1], boundaries[0])
  AD = np.subtract(boundaries[3], boundaries[0])

  return (0 < np.dot(AM, AB) < np.dot(AB, AB)) and (0 < np.dot(AM, AD) < np.dot(AD, AD))

In [ ]:
is_inside([[-125,30],[-115,30],[-115,55],[-125,55],[-125,30]], [-117, 40])

True

In [ ]:
def getLatLon(lat,lon):
  return 55-lat/4,125-lon/4
def getGridIndex(x,y):
  return (4*(55-x)).round(), 4*(y+125).round()

flash_lat = df['event_lat'].values
flash_lon = df['event_lon'].values
flash_latlon = np.column_stack((flash_lat, flash_lon))

lightning_grid = np.zeros((101, 41))
for x in range(flash_latlon.shape[0]):
  tempx,tempy = getGridIndex(flash_latlon[x][0],flash_latlon[x][1])
  try:
    lightning_grid[tempx][tempy]+=1
    print(flash_latlon[x][0].round(),flash_latlon[x][1].round())
    print(tempx,tempy)
  except IndexError:
    pass


In [ ]:
flash_latlon = np.column_stack((flash_lat, flash_lon))
print(flash_latlon)

[[ -66.56076546 -141.69922301]
 [ -66.56076546 -141.69922301]
 [ -66.56076546 -141.69922301]
 ...
 [ -66.52060201 -141.75966622]
 [ -66.52043697 -141.75969511]
 [ -66.52059789 -141.75983539]]
